In [2]:
import pandas as pd
!conda install -c conda-forge wikipedia --yes
import wikipedia as wp

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - wikipedia


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    wikipedia-1.4.0            |             py_2          13 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

    wikipedia:       1.4.0-py_2        conda-forge

The following packages will be UPDATED:

    ca-certificates: 2019.8.28-0                   --> 2019.9.11-hecc5488_0 conda-forge
    certifi:         2019.9.11-py36_0       

## read the page from wikipedia

In [3]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header=0)[0]

## drop the row with Borough equals Not assigned

In [4]:
df_assigned =df[df.Borough != "Not assigned"]
df_assigned.reset_index(drop = True, inplace=True)
df_assigned.head(20)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Meger rows with the same postcode and set 'Not assigned' neighborhood to be equal to Borough

In [5]:
df_assigned = df_assigned.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ','.join(set(x)))
na_nb_idx =df_assigned['Neighbourhood'] == 'Not assigned'
df_assigned.loc[na_nb_idx, 'Neighbourhood'] = df_assigned[na_nb_idx].Borough
df_assigned[df_assigned['Postcode'] == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


## show the shape of the dataset

In [6]:
df_assigned.shape

(103, 3)

## Add latitude and longitude info for each neighborhood

In [7]:
import numpy as np
df_assigned['Latitude'] = np.nan
df_assigned['Longitude'] = np.nan

In [8]:
df_assigned.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",NaN,NaN
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",NaN,NaN
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",NaN,NaN
3,M1G,Scarborough,Woburn,NaN,NaN
4,M1H,Scarborough,Cedarbrae,NaN,NaN


In [9]:
!wget -O GeoCord.csv http://cocl.us/Geospatial_data/

--2019-11-04 09:33:35--  http://cocl.us/Geospatial_data/
Resolving cocl.us (cocl.us)... 161.202.50.39
Connecting to cocl.us (cocl.us)|161.202.50.39|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/ [following]
--2019-11-04 09:33:35--  https://cocl.us/Geospatial_data/
Connecting to cocl.us (cocl.us)|161.202.50.39|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-04 09:33:36--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 103.116.4.197
Connecting to ibm.box.com (ibm.box.com)|103.116.4.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-04 09:33:36--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1

In [10]:
df_cord = pd.read_csv('GeoCord.csv') # Read the csv data
df_cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# For each postcode in df_assigned, find corresponding coordinates in df_cord and assign it to df_assigned
for idx in df_assigned.index:
    cord_idx = df_cord['Postal Code'] == df_assigned.loc[idx, 'Postcode']
    df_assigned.at[idx, 'Latitude'] = df_cord.loc[cord_idx, 'Latitude'].values
    df_assigned.at[idx, 'Longitude'] = df_cord.loc[cord_idx, 'Longitude'].values

# Display 
df_assigned.head(20)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Morningside,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


## toronto map

In [17]:
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium --yes
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

geolocator = Nominatim(user_agent="toronta_cousera")
address = 'Toronto'
try:
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))
except AttributeError:
    print('Cannot find: {}, will drop index: {}'.format(address, index))

my_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_assigned['Latitude'], df_assigned['Longitude'], df_assigned['Postcode']):
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(my_map)  
    
my_map

The geograpical coordinates of Toronto are 43.653963, -79.387207.
